In [297]:
import trimesh
import numpy as np
import pandas as pd

In [298]:
mesh = trimesh.load('bunny.obj')
triangles = mesh.triangles

In [299]:
AREA_THRESHOLD = 0.000004
Z_THRESHOLD = 0.011

area_rule_inactive = True
angle_rule_inactive = False
z_rule_inactive = False

In [300]:
import ipywidgets as widgets

max_zet = 10 
min_zet = 0


list_of_toggles = []
list_of_sliders = []
for rule in ["Area","Angles","Zthreshold"]:
    toggle = widgets.ToggleButton(
        value=False,
        description=rule,
        disabled=False,
        button_style='',
        tooltip='Description',
        icon='check' 
    )

    slider = widgets.FloatSlider(
    value=(max_zet - min_zet)/2,
    min=min_zet,
    max=max_zet,
    step=0.5,
    description=rule,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    )

    list_of_toggles.append(toggle)
    list_of_sliders.append(slider)
widgets.VBox([widgets.HBox(list_of_toggles),widgets.HBox(list_of_sliders)])

In [301]:
def split_trianges(triangle, bary):
    new_triangles = np.empty((3,3,3), dtype=float, order='C')
    for i in range(3):
        new_triangle = numpy.array(triangle)
        new_triangle[i,:] = np.array(bary)
        new_triangles[i,:,:] = new_triangle
    return new_triangles

In [302]:
triangle_data = pd.DataFrame(
    {'triangles': triangles.tolist(),
     'areas': trimesh.triangles.area(triangles=triangles),
     'angles': np.degrees(trimesh.triangles.angles(triangles)).tolist(),
     'barycenters': mesh.triangles_center.tolist(),
     'Zs': triangles[:,:,-1].tolist(),
    })

triangle_data["angles_min"] = triangle_data["angles"].apply(min)
triangle_data["z_max"] = triangle_data["Zs"].apply(max)
triangle_data['area_rule'] =  triangle_data["areas"] > AREA_THRESHOLD
triangle_data['angle_rule'] =  triangle_data["angles_min"] > 45
triangle_data['z_rule'] = triangle_data["z_max"] < Z_THRESHOLD
triangle_data['split'] =  np.where((triangle_data['area_rule'] | area_rule_inactive) & (triangle_data['angle_rule'] | angle_rule_inactive ) & (triangle_data['z_rule'] | z_rule_inactive), True, False)

triangle_data["triangles"] = triangle_data.apply(lambda x: split_trianges(x.triangles, x.barycenters), axis=1).where(triangle_data.split,triangle_data.triangles)




In [303]:
triangle_data

,triangles,areas,angles,barycenters,Zs,angles_min,z_max,area_rule,angle_rule,z_rule,split
0,"[[[-0.03549733333333333, 0.10634066666666668, ...",0.000002,"[68.65010727572552, 51.003104362218174, 60.346...","[-0.03549733333333333, 0.10634066666666668, -0...","[-0.020165, -0.020017, -0.020533]",51.003104,-0.020017,False,True,True,True
1,"[[-0.086198, 0.109048, 0.009768], [-0.086102, ...",0.000001,"[103.58977538510015, 46.1996332827074, 30.2105...","[-0.08622333333333333, 0.10844666666666668, 0....","[0.009768, 0.009, 0.011546]",30.210591,0.011546,False,False,False,False
2,"[[-0.013148, 0.129073, 0.007432], [-0.015074, ...",0.000007,"[79.77671329816263, 56.78819874396889, 43.4350...","[-0.012889, 0.12926666666666667, 0.00929300000...","[0.007432, 0.010126, 0.010321]",43.435088,0.010321,True,False,True,False
3,"[[[-0.013092, 0.12889566666666666, 0.005830333...",0.000005,"[47.419357912338334, 83.06818196468015, 49.512...","[-0.013092, 0.12889566666666666, 0.00583033333...","[0.00494, 0.007432, 0.005119]",47.419358,0.007432,True,True,True,True
4,"[[-0.018945, 0.127729, 0.015572], [-0.022487, ...",0.000005,"[58.08640600556322, 53.052778650224695, 68.860...","[-0.02064433333333333, 0.127018, 0.01640066666...","[0.015572, 0.015442, 0.018188]",53.052779,0.018188,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
27859,"[[-0.048533, 0.034286, 0.016076], [-0.05171, 0...",0.000004,"[22.891687392537257, 67.56677078939136, 89.541...","[-0.050882666666666666, 0.033794, 0.017689]","[0.016076, 0.01932, 0.017671]",22.891687,0.019320,False,False,False,False
27860,"[[-0.048917, 0.034473, 0.01337], [-0.052405, 0...",0.000003,"[10.721874974617826, 69.33417355153718, 99.943...","[-0.05141266666666666, 0.03382366666666667, 0....","[0.01337, 0.017671, 0.016747]",10.721875,0.017671,False,False,False,False
27861,"[[-0.04418, 0.03537, 0.010712], [-0.048917, 0....",0.000005,"[75.01910849618159, 20.39180981538891, 84.5890...","[-0.046204, 0.035033999999999996, 0.0111363333...","[0.010712, 0.01337, 0.009327]",20.391810,0.013370,True,False,False,False
27862,"[[-0.048917, 0.034473, 0.01337], [-0.052916, 0...",0.000002,"[144.6999201416305, 7.103629154064933, 28.1964...","[-0.050142, 0.03423366666666667, 0.01405466666...","[0.01337, 0.016747, 0.012047]",7.103629,0.016747,False,False,False,False
